## Phase II:
#### The s3 folder system will be date wise for eg: “09-25-2021”, “09-26-2021”. (Future dates) [Manual]
* Each folder will have data equivalent to 1 million
* Create a cron job/scheduler/airflow that will read today’s date folder and index to Elasticsearch.* 
* Create an API using python that receives the parameters called date and copies the given date folder to Elasticsearch. Eg: (http://localhost:8888/data-ingest?date=09-25-2021)


In [1]:
!pip install smart_open

In [1]:
import os
import pandas as pd
from smart_open import smart_open

try:
    import boto3
except Exception as e:
    print(e)
with open("secret.txt","r") as f:
    data = f.readlines()
    aws_key=data[0].strip("\n")
    aws_secret=data[1].strip("\n")



In [2]:
client=boto3.client("s3",
                   aws_access_key_id=aws_key,
                   aws_secret_access_key=aws_secret,
                   region_name="us-west-1")

In [44]:
from datetime import date
today = date.today()
# Current date time in local system
date=today.strftime("%-m-%-d-%Y")

In [45]:
paginator=client.get_paginator('list_objects_v2')
pages=paginator.paginate(Bucket='suyesh-es-task-26102021',Prefix=date+'/')

In [48]:
tmp=[]
for page in pages:
    for obj in page['Contents']:
        if(obj['Key'].endswith('.json')):
            tmp.append(obj['Key'])


'11-3-2021/part-00000-614cf801-8141-4f35-aeb7-472cd6b25fdb-c000.json'

In [3]:
bucket_name = 'suyesh-es-task-26102021'
object_key ='11-4-2021/part-00000-680cf244-5c4f-4fd4-a216-b1aa996d01e3-c000.json'

In [ ]:
path = 's3://{}:{}@{}/{}'.format(aws_key, aws_secret, bucket_name, object_key)

df = pd.read_json(smart_open(path), lines=True)
df.head()

In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch import helpers

In [2]:
# Elastic search configuation
ENDPOINT="http://localhost:9200/"
es=Elasticsearch(timeout=600,hosts=ENDPOINT)


In [59]:
df = df.where(pd.notnull(df), None)

In [60]:
documents = df.to_dict(orient='records')


In [61]:
len(documents)

1000000

In [62]:
documents[0]

{'category_id': 2053013552259662037,
 'event_time': Timestamp('2019-10-01 16:56:42+0000', tz='UTC'),
 'event_type': 'view',
 'price': 224.19,
 'product_id': 8500421,
 'user_id': 526731152,
 'user_session': 'c5889654-8f52-4ea8-9429-81d4bf998670',
 'category_code': None,
 'brand': None}

In [3]:
settings={
    "mappings" : {
      "properties" : {
        "id" : {
          "type" : "long"
        },
        "timestamp" : {
          "type" : "date"
        },
        "brand" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "category_code" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "category_id" : {
          "type" : "long"
        },
        "event_time" : {
          "type" : "date"
        },
        "event_type" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "price" : {
          "type" : "float"
        },
        "product_id" : {
          "type" : "long"
        },
        "user_id" : {
          "type" : "long"
        },
        "user_session" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        }
      }
    }
  
}

In [4]:
es.indices.create(index='ecommerce',ignore=400,body=settings)

/tmp/ipykernel_4803/166438481.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index='ecommerce',ignore=400,body=settings)
/home/suyesh/.local/lib/python3.9/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'ecommerce'}

In [20]:
count=int(es.cat.count('ecommerce', params={"format": "json"})[0]['count'])
body={
   "size": 1,
   "sort": { "id": "desc"},
   "query": {
      "match_all": {}
   }
}
if(count!=0):
    res = es.search(index="ecommerce", body=body)
    i=int(res['hits']['hits'][0]["_source"]['id'])+1
else:
    i=0
    

/tmp/ipykernel_20949/677982156.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  count=int(es.cat.count('ecommerce', params={"format": "json"})[0]['count'])
/tmp/ipykernel_20949/677982156.py:10: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="ecommerce", body=body)


In [21]:
i

1000000

In [8]:
#defining schma for ecommerce
def ecommerce(i):

    print(i)

In [67]:
#defining schma for ecommerce
def ecommerce(df,i):
    print(i)
    for line in df:
        if(line.get('brand')== None):
            brand=None
        else:
            brand=line.get('brand')
            
        if(line.get('category_code')== None):
            category_code=None
        else:
            category_code=line.get('category_code')

        yield{
            '_index':'ecommerce',
            '_type':'_doc',
            '_id':i,
            '_source':{
                'brand':brand,
                'category_id':line.get('category_id',''),
                'event_time':line.get('event_time',''),
                'event_type':line.get('event_type',''),
                'price':line.get('price',''),
                'product_id':line.get('product_id',''),
                'user_id':line.get('user_id',''),
                'user_session':line.get('user_session',''),
                'category_code':category_code

            }
        
            

        }
        i=i+1
    print(i)

In [9]:
# attempt to index the dictionary entries using the helpers.bulk() method
try:
    print ("\nAttempting to index the list of docs using helpers.bulk()")

    # use the helpers library's Bulk API to index list of Elasticsearch docs
    resp = helpers.bulk(\
    es,\
    ecommerce(documents,i),\
    index = "ecommerce",\
    doc_type = "_doc"
    )
    print("done")

except Exception as err:
    print(err)

1000000


/tmp/ipykernel_6224/845805730.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  int(es.cat.count('bla', params={"format": "json"})[0]['count'])


0

/tmp/ipykernel_6224/4175245481.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="ecommerce", body=body)
/home/suyesh/.local/lib/python3.9/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Loading the fielddata on the _id field is deprecated and will be removed in future versions. If you require sorting or aggregating on this field you should also include the id in the body of your documents, and map this field as a keyword field that has [doc_values] enabled
  warnings.warn(message, category=ElasticsearchWarning)


'999999'